# Gesamtprogramm

In [1]:
import ImageHandler as ih
import numpy as np
import pandas as pd
import random
from ImageHandler import cv
from pprint import pprint

loader = ih.ImageLoader('bilddaten\Dosenoeffner')
loader.AddDirRealPath('bilddaten\Flaschenoeffner')
loader.AddDirRealPath('bilddaten\Korkenzieher')

# loader.AddDirRealPath('train\Dosenoeffner')
# loader.AddDirRealPath('train\Flaschenoeffner')
# loader.AddDirRealPath('train\Korkenzieher')

# loader.AddDirRealPath('iliasdaten\Dosenoeffner')
# loader.AddDirRealPath('iliasdaten\Flaschenoeffner')
# loader.AddDirRealPath('iliasdaten\Korkenzieher')

Added Directory bilddaten\Dosenoeffner
Added Directory bilddaten\Flaschenoeffner
Added Directory bilddaten\Korkenzieher


In [2]:
loader.MakeSampleEntrys(3)

print('dateilänge = ',len(loader.allImgPaths))
print('Randomlänge = ',len(loader.sampleEntrysPath))

changer = ih.ImageChanger(loader.sampleEntrysPath, 0, False)

dateilänge =  767
Randomlänge =  3


In [3]:

changer.FrameByWidth(400)
changer.BilateralFilter(10,10)
# changer.ShowAll()

changer.Normieren()
changer.ColorAdapt(sigma=1)
# changer.ShowAll()

changer.BrightTresh()
# changer.ShowAll()

changer.Bordering(100)
# changer.ShowAll()

changer.InvertCheck()
# changer.ShowAll()

changer.Dilation(7,2)
# changer.ShowAll()

changer.Erode(7,2)
# changer.ShowAll()

changer.ConturMerkmale(type1=cv.RETR_TREE,type2=cv.CHAIN_APPROX_NONE,epsyValue=0.005)
# changer.ShowAll()
changer.PrintErrors(all=True)
changer.PrintValuesToCSV('extracsvAll1.csv')

Total Number of Errors = 0
-------------Values CSV Done------------


In [4]:
df = pd.read_csv("extracsvAll.csv")

label_name = "label"
df = df.rename(columns={"type":label_name})
df.head()

,approxAnzahl,areaRatio,circleLike,contureDistance,conturenAnzahl,farPoint,keyPointsAnzahl,mean_val,middlepointDistance,minimal_ratio,minimal_ratio2,momentpointDistance,rectangleLike,seradity,seradity2,solidity,solidity2,steiner,label
0,26,0.004413,0.379250,1.003808,3,65806,434,254.011059,-2.810088,2.710155,1.045454,-8.246211,1.020743,0.554420,0.970142,0.669933,0.978365,5.099020,Dosenoeffner
1,27,0.062218,0.442046,1.073125,2,29660,432,240.198330,-0.998737,2.002529,1.673469,24.331050,1.249528,0.650614,0.954057,0.499288,0.978225,30.232433,Korkenzieher
2,20,0.109854,0.498771,1.251264,2,7145,418,228.538098,7.272945,3.037383,1.585366,23.769729,1.516755,0.910871,0.951853,0.739691,0.964045,16.278821,Flaschenoeffner
3,20,0.013830,0.253412,1.522871,3,27976,322,249.384412,7.473587,1.616767,1.802632,-0.000000,0.277719,0.761132,0.958875,0.718056,0.963532,17.720045,Dosenoeffner
4,19,0.008913,0.401001,1.051924,2,56244,407,252.974080,4.510240,2.314801,1.208333,-7.615773,1.008971,0.670125,0.964847,0.648434,0.975986,19.235384,Dosenoeffner


In [5]:
#df = df.drop("approxAnzahl",axis=1)
#df = df.drop("areaRatio",axis=1)
#df = df.drop("circleLike",axis=1)
#df = df.drop("contureDistance",axis=1)
#df = df.drop("conturenAnzahl",axis=1)
#df = df.drop("farPoint",axis=1)
#df = df.drop("keyPointsAnzahl",axis=1)
#df = df.drop("mean_val",axis=1)
#df = df.drop("middlepointDistance",axis=1)
#df = df.drop("minimal_ratio",axis=1)
#df = df.drop("minimal_ratio2",axis=1)
#df = df.drop("momentpointDistance",axis=1)
#df = df.drop("rectangleLike",axis=1)
#df = df.drop("seradity",axis=1)
#df = df.drop("seradity2",axis=1)
#df = df.drop("solidity",axis=1)
#df = df.drop("solidity2",axis=1)
#df = df.drop("steiner",axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   approxAnzahl         752 non-null    int64  
 1   areaRatio            752 non-null    float64
 2   circleLike           752 non-null    float64
 3   contureDistance      752 non-null    float64
 4   conturenAnzahl       752 non-null    int64  
 5   farPoint             752 non-null    int64  
 6   keyPointsAnzahl      752 non-null    int64  
 7   mean_val             752 non-null    float64
 8   middlepointDistance  752 non-null    float64
 9   minimal_ratio        752 non-null    float64
 10  minimal_ratio2       752 non-null    float64
 11  momentpointDistance  752 non-null    float64
 12  rectangleLike        752 non-null    float64
 13  seradity             752 non-null    float64
 14  seradity2            752 non-null    float64
 15  solidity             752 non-null    flo

In [6]:
dec = ih.Dectree()
train_df, test_df = dec.frame_split(df, test_size_in_percent=20)

In [7]:
tree = dec.make_decision_tree(train_df,min_samples=1, max_depth=15)
print(tree)

{'seradity <= 0.8487558929054093': [{'farPoint <= 44219.0': [{'minimal_ratio <= 2.149693097787739': [{'rectangleLike <= 0.2800011909088761': [{'steiner <= 49.36930794957768': ['Dosenoeffner', 'Korkenzieher']}, {'mean_val <= 249.8407039259926': [{'momentpointDistance <= 16.513878188659973': [{'minimal_ratio <= 1.494639781131887': ['Korkenzieher', 'Flaschenoeffner']}, {'minimal_ratio <= 1.572040202031876': [{'contureDistance <= 1.0735527031424197': ['Flaschenoeffner', {'middlepointDistance <= 2.6711876004388095': ['Flaschenoeffner', 'Korkenzieher']}]}, 'Korkenzieher']}]}, {'steiner <= 10.547942460873601': ['Dosenoeffner', 'Korkenzieher']}]}]}, {'middlepointDistance <= 15.934968714676149': [{'rectangleLike <= 1.149788854095888': [{'middlepointDistance <= 15.025139566903114': [{'rectangleLike <= 1.1327089062906888': ['Dosenoeffner', {'steiner <= 18.120619473869912': ['Dosenoeffner', 'Korkenzieher']}]}, {'solidity <= 0.785494452142694': ['Korkenzieher', 'Dosenoeffner']}]}, {'farPoint <= 140

In [8]:
accuracy, keynumbers = dec.calculate_keynumbers(test_df, tree, label_name)
accuracy

0.9133333333333333

In [9]:
keynumbers = pd.DataFrame(keynumbers)
keynumbers

,Label,recall,precision,f1score
0,Dosenoeffner,0.870968,0.915254,0.892562
1,Flaschenoeffner,0.767857,1.000000,0.868687
2,Korkenzieher,0.888889,0.833333,0.860215


# Bayes

In [10]:
bay = ih.Bayes()

df_bayes = df.round(1)
df_train, df_test = bay.frame_split(df_bayes, test_size_in_percent=20)
test_labels = df_test[label_name]
df_test.drop(label_name,axis=1)
print(df_test)
print(df_train)
print(test_labels)

     approxAnzahl  areaRatio  circleLike  contureDistance  conturenAnzahl  \
710            12        0.3         0.5              0.8               2   
49             11        0.0         0.4              1.3               2   
80             18        0.1         0.3              0.9               2   
602            16        0.1         0.5              1.2               2   
163            43        0.0         0.5              1.4               2   
..            ...        ...         ...              ...             ...   
669            34        0.0         0.4              1.0               4   
160            12        0.1         0.1              1.0               2   
630            22        0.2         0.4              1.2               3   
254            34        0.1         0.1              1.7              21   
522            27        0.0         0.4              1.0               7   

     farPoint  keyPointsAnzahl  mean_val  middlepointDistance  minimal_rati

In [11]:
bayes_table = bay.wahrscheinlichkeiten(df_train, label_column=label_name)
pprint(bayes_table)

{'approxAnzahl': {4: array([0.01652893, 0.08370044, 0.04310345]),
                  6: array([0.00413223, 0.01762115, 0.00431034]),
                  8: array([0.00413223, 0.02643172, 0.00431034]),
                  9: array([0.00413223, 0.00881057, 0.00431034]),
                  11: array([0.01239669, 0.030837  , 0.00431034]),
                  12: array([0.02066116, 0.06167401, 0.00431034]),
                  13: array([0.03305785, 0.03524229, 0.00862069]),
                  14: array([0.00413223, 0.05286344, 0.00862069]),
                  15: array([0.02479339, 0.04405286, 0.01293103]),
                  16: array([0.02892562, 0.0660793 , 0.01724138]),
                  17: array([0.03719008, 0.05726872, 0.00862069]),
                  18: array([0.10743802, 0.07929515, 0.02586207]),
                  19: array([0.04958678, 0.04405286, 0.02155172]),
                  20: array([0.09917355, 0.04405286, 0.04741379]),
                  21: array([0.08677686, 0.03964758, 0.05172414]),

In [12]:
accuracy, keynumbers = bay.calculate_keynumbers(test_df, bayes_table, label_name)
accuracy

0.88

In [13]:
keynumbers = pd.DataFrame(keynumbers)
keynumbers

,Label,recall,precision,f1score
0,Dosenoeffner,0.825397,0.881356,0.852459
1,Flaschenoeffner,0.727273,0.930233,0.816327
2,Korkenzieher,0.800000,0.833333,0.816327
